# Multi-Regime Climate-Financial Risk Transmission Engine
## Example Usage Notebook

This notebook demonstrates the key features of the Climate-Financial Risk Transmission Engine.

In [ ]:
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from src.data_ingestion import FinancialDataCollector, ClimateDataCollector, DataProcessor
from src.econometric_modeling import MarkovRegimeSwitching, VARModel, CausalityAnalyzer
from src.mathematical_finance import JumpDiffusionModel

plt.style.use('seaborn-v0_8')
%matplotlib inline

## 1. Data Collection

In [ ]:
# Initialize data collectors
financial_collector = FinancialDataCollector()
climate_collector = ClimateDataCollector()

# Collect financial data
print("Collecting financial data...")
financial_data = financial_collector.get_comprehensive_data()

# Collect climate data
print("Collecting climate data...")
climate_data = climate_collector.get_comprehensive_climate_data()

print("Data collection complete!")

## 2. Data Preprocessing

In [ ]:
# Initialize data processor
processor = DataProcessor()

# Process financial data
if not financial_data['indices'].empty:
    clean_financial = processor.clean_financial_data(financial_data['indices'])
    returns = processor.calculate_returns(clean_financial['Close'])
    
    print(f"Financial data shape: {clean_financial.shape}")
    print(f"Returns shape: {returns.shape}")
    
    # Plot returns
    plt.figure(figsize=(12, 6))
    returns.plot()
    plt.title('Financial Returns')
    plt.ylabel('Returns')
    plt.show()

## 3. Regime Detection

In [ ]:
# Markov Regime-Switching Model
regime_model = MarkovRegimeSwitching(n_regimes=2)

# Simulate some data for demonstration
np.random.seed(42)
n_obs = 500
regime_1_data = np.random.normal(0.001, 0.02, n_obs//2)
regime_2_data = np.random.normal(-0.005, 0.05, n_obs//2)
demo_data = np.concatenate([regime_1_data, regime_2_data])

# Fit the model
print("Fitting regime-switching model...")
regime_model.fit(demo_data)

# Get regime probabilities
regime_probs = regime_model.regime_probabilities_over_time(demo_data)

# Plot results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Plot data
ax1.plot(demo_data)
ax1.set_title('Time Series Data')
ax1.set_ylabel('Value')

# Plot regime probabilities
ax2.plot(regime_probs)
ax2.set_title('Regime Probabilities')
ax2.set_ylabel('Probability')
ax2.set_xlabel('Time')
ax2.legend(['Regime 0', 'Regime 1'])

plt.tight_layout()
plt.show()

# Print regime statistics
print("\nRegime Statistics:")
regime_stats = regime_model.get_regime_statistics()
for regime, stats in regime_stats.items():
    print(f"{regime}: Mean={stats['mean']:.4f}, Std={stats['std']:.4f}")

## 4. Granger Causality Analysis

In [ ]:
# Causality analyzer
causality_analyzer = CausalityAnalyzer()

# Create some demo data
np.random.seed(42)
n_obs = 200
climate_variable = np.random.normal(0, 1, n_obs)
financial_variable = np.zeros(n_obs)

# Create causality: financial variable depends on lagged climate variable
for t in range(1, n_obs):
    financial_variable[t] = 0.3 * climate_variable[t-1] + 0.5 * financial_variable[t-1] + np.random.normal(0, 0.5)

# Test causality
print("Testing Granger causality...")
causality_result = causality_analyzer.granger_test(climate_variable, financial_variable, lags=1)

print(f"\nGranger Causality Test Results:")
print(f"F-statistic: {causality_result['f_statistic']:.4f}")
print(f"P-value: {causality_result['p_value']:.4f}")
print(f"Significant: {causality_result['significant']}")

# Plot the variables
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.plot(climate_variable)
plt.title('Climate Variable')
plt.ylabel('Value')

plt.subplot(2, 1, 2)
plt.plot(financial_variable)
plt.title('Financial Variable')
plt.ylabel('Value')
plt.xlabel('Time')

plt.tight_layout()
plt.show()

## 5. Jump-Diffusion Modeling

In [ ]:
# Jump-diffusion model
jump_model = JumpDiffusionModel(
    mu=0.05,           # drift
    sigma=0.2,         # volatility
    jump_intensity=0.1, # jump frequency
    jump_mean=-0.05,   # average jump size
    jump_std=0.1       # jump size volatility
)

# Simulate climate events
np.random.seed(42)
n_steps = 252
climate_events = np.random.binomial(1, 0.05, n_steps)  # 5% chance of climate event per day

# Simulate paths
print("Simulating jump-diffusion paths...")
paths = jump_model.simulate_multiple_paths(
    T=1.0,                    # 1 year
    steps=n_steps,           # daily steps
    n_paths=1000,            # number of simulations
    S0=100.0,                # initial value
    climate_events=climate_events
)

# Plot results
plt.figure(figsize=(12, 8))

# Plot first 50 paths
plt.subplot(2, 2, 1)
for i in range(50):
    plt.plot(paths[i], alpha=0.3, color='blue')
plt.title('Sample Paths')
plt.ylabel('Price')

# Plot climate events
plt.subplot(2, 2, 2)
plt.plot(climate_events, 'ro', markersize=3)
plt.title('Climate Events')
plt.ylabel('Event Indicator')

# Plot final price distribution
plt.subplot(2, 2, 3)
final_prices = paths[:, -1]
plt.hist(final_prices, bins=50, alpha=0.7, density=True)
plt.title('Final Price Distribution')
plt.xlabel('Price')
plt.ylabel('Density')

# Plot returns distribution
plt.subplot(2, 2, 4)
returns = (final_prices - 100) / 100
plt.hist(returns, bins=50, alpha=0.7, density=True)
plt.title('Returns Distribution')
plt.xlabel('Return')
plt.ylabel('Density')

plt.tight_layout()
plt.show()

# Print statistics
print(f"\nSimulation Statistics:")
print(f"Final price mean: {np.mean(final_prices):.2f}")
print(f"Final price std: {np.std(final_prices):.2f}")
print(f"Average return: {np.mean(returns):.4f}")
print(f"Return volatility: {np.std(returns):.4f}")

## 6. Risk Metrics

In [ ]:
# Calculate VaR using jump-diffusion model
print("Calculating Value at Risk...")
var_results = jump_model.calculate_var(
    S0=100.0,
    T=1.0,
    confidence_level=0.05,  # 95% VaR
    n_simulations=10000,
    climate_events=climate_events
)

print(f"\nVaR Results (95% confidence):")
print(f"VaR (absolute): {var_results['var_absolute']:.2f}")
print(f"VaR (relative): {var_results['var_relative']:.4f}")
print(f"Expected Shortfall: {var_results['expected_shortfall']:.4f}")
print(f"Minimum return: {var_results['min_return']:.4f}")
print(f"Maximum return: {var_results['max_return']:.4f}")

# Option pricing example
print("\nCalculating option prices...")
option_results = jump_model.calculate_option_price_mc(
    S0=100.0,
    K=100.0,      # at-the-money
    T=0.25,       # 3 months
    r=0.05,       # risk-free rate
    option_type='call',
    n_simulations=50000
)

print(f"\nOption Pricing Results:")
print(f"Option price: {option_results['option_price']:.4f}")
print(f"Standard error: {option_results['std_error']:.4f}")
print(f"95% CI: [{option_results['confidence_interval_95'][0]:.4f}, {option_results['confidence_interval_95'][1]:.4f}]")

## 7. Model Comparison

In [ ]:
# Compare models with and without climate effects
print("Comparing models with and without climate effects...")

# Model without climate effects
normal_model = JumpDiffusionModel(
    mu=0.05, sigma=0.2, jump_intensity=0.05,
    jump_mean=-0.02, jump_std=0.05
)

# Model with climate effects
climate_model = JumpDiffusionModel(
    mu=0.05, sigma=0.2, jump_intensity=0.1,
    jump_mean=-0.05, jump_std=0.1
)

# Simulate paths
normal_paths = normal_model.simulate_multiple_paths(T=1, steps=252, n_paths=1000, S0=100)
climate_paths = climate_model.simulate_multiple_paths(T=1, steps=252, n_paths=1000, S0=100, climate_events=climate_events)

# Compare distributions
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
normal_returns = (normal_paths[:, -1] - 100) / 100
climate_returns = (climate_paths[:, -1] - 100) / 100

plt.hist(normal_returns, bins=50, alpha=0.7, label='Normal Model', density=True)
plt.hist(climate_returns, bins=50, alpha=0.7, label='Climate Model', density=True)
plt.title('Return Distributions')
plt.xlabel('Return')
plt.ylabel('Density')
plt.legend()

plt.subplot(1, 2, 2)
# Plot some sample paths
for i in range(10):
    plt.plot(normal_paths[i], alpha=0.5, color='blue')
    plt.plot(climate_paths[i], alpha=0.5, color='red')
plt.title('Sample Paths Comparison')
plt.xlabel('Time')
plt.ylabel('Price')

plt.tight_layout()
plt.show()

# Print comparison statistics
print(f"\nModel Comparison:")
print(f"Normal Model - Mean return: {np.mean(normal_returns):.4f}, Std: {np.std(normal_returns):.4f}")
print(f"Climate Model - Mean return: {np.mean(climate_returns):.4f}, Std: {np.std(climate_returns):.4f}")
print(f"Difference in volatility: {np.std(climate_returns) - np.std(normal_returns):.4f}")

## Summary

This notebook demonstrated the key capabilities of the Multi-Regime Climate-Financial Risk Transmission Engine:

1. **Data Collection**: Gathered financial and climate data from free sources
2. **Regime Detection**: Identified different market regimes using Markov switching models
3. **Causality Analysis**: Tested relationships between climate and financial variables
4. **Jump-Diffusion Modeling**: Simulated price paths with climate-triggered jumps
5. **Risk Metrics**: Calculated VaR and option prices
6. **Model Comparison**: Showed the impact of climate effects on risk metrics

The platform provides a comprehensive framework for analyzing climate-financial risk transmission across different market regimes.